In [ ]:
#!g1.1
import pandas as pd
df = pd.read_csv('cert_0.csv')

In [663]:
#!g1.1
df2 = df.sort_values(by=['reasons'], ascending=False, axis=0)
df2 = df2.fillna(0)
df2['weight'] = df2['weight'].apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0)
df2['base_weight'] = df2['base_weight'].apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0)

In [664]:
#!g1.1
df2.head()

In [665]:
#!g1.1
 df2 = df2.drop(['Unnamed: 0'],1)

In [661]:
#!g1.1
import os

import tensorflow as tf
from tensorflow.keras.datasets import mnist
import tensorflow.keras as k
from tensorflow.keras import models
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing


# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#     pass

In [666]:
#!g1.1
df2 = df2.drop(['canceled_cert_date'], axis=1).astype({'consignee_be_sub_region_id': str, 'consignee_ent_sub_region_id': str, 'consignor_be_sub_region_id': str, 'consignor_ent_sub_region_id': str, 'guidRecipientCountry': str, 'repaid_cert_date': str})
dataset = df_to_dataset(df2)

In [667]:
#!g1.1
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('reasons')
  print(labels)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), to_categorical(labels, num_classes=9)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [668]:
#!g1.1
import numpy as mp
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('Status id:', train_features['cert_status_id'])
print('A batch of targets:', label_batch)

In [669]:
#!g1.1
def get_normalization_layer(name, dataset):
    
  normalizer = preprocessing.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)

  return normalizer

In [670]:
#!g1.1
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  return lambda feature: encoder(index(feature))


In [671]:
#!g1.1
batch_size = 10
test_ds = df_to_dataset(dataset, shuffle=False, batch_size=batch_size)

In [672]:
#!g1.1
all_inputs = []
encoded_features = []

for header in ['cert_type_id', 'cert_vetform_id', 'cert_status_id',
       'cert_nature_type_id', 'cert_request_type_id', 'cert_reqsource_type_id',
       'consignor_be_id', 'consignor_ent_id', 'consignee_be_id',
       'consignee_ent_id', 'sub_product_id', 'product_id', 'doctor_id',
       'unit_id', 'base_unit_id', 'consignor_be_region_id',
       'consignor_ent_region_id', 'consignee_be_region_id',
       'consignee_ent_region_id', 'transfer_type_id', 'product_type_id',
       'cert_source_id', 'cert_protected_id', 'repaid_doctor_id',
       'canceled_doctor_id', 'vetExpertise', 'product_name_id', 'former_id',
       'protocol_version', 'transit_time_hour', 'weight', 'base_weight']:
    
  print(header)
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)


categorical_cols = ['unit_guid', 'cert_date', 'cert_insert_date',
       'consignor_be_sub_region_id', 'consignee_be_sub_region_id',
       'consignor_ent_sub_region_id', 'consignee_ent_sub_region_id',
       'repaid_cert_date', 'guidOriginCountry',
       'guidRecipientCountry']
for header in categorical_cols:
  print(header)
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string')
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

In [674]:
#!g1.1
#!g1.1
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(128, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(9)(x)
model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [675]:

model.load_weights('Weights/weights')

loss, accuracy = model.evaluate(dataset)
print("Accuracy", accuracy)

In [ ]:
df2 = df.head(90) + df.sort_values(by=['reasons'], ascending=False, axis=0)
df2.write_csv(git.csv)